If cuda is not working first try reinstalling torch and restarting computer, if this does not work you are fucked yet again

In [1]:

import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device
torch.cuda.is_available()

True

In [ ]:
# Import necessary libraries
from top2vec import Top2Vec


## top2vec on news data

In [1]:
import pandas as pd
ROOT_PATH = '/home/jhladnik/'

NUM_SAMPLES = 5000
df = pd.read_parquet(
    f'{ROOT_PATH}/data/eventregistry/df_news_lemmas_{NUM_SAMPLES}.parquet.gzip')


In [2]:
df["lemmatized_text_string"] = df.lemmatized_text.apply(lambda x: " ".join(x))

In [5]:
# 3 min on remote  for 5k samples
model = Top2Vec(documents=list(df["lemmatized_text_string"]), speed="learn", workers=24)


2023-01-24 20:19:09,604 - top2vec - INFO - Pre-processing documents for training
2023-01-24 20:19:11,973 - top2vec - INFO - Creating joint document/word embedding
2023-01-24 20:21:49,926 - top2vec - INFO - Creating lower dimension embedding of documents
2023-01-24 20:22:16,130 - top2vec - INFO - Finding dense areas of documents
2023-01-24 20:22:16,262 - top2vec - INFO - Finding topics


In [6]:
topic_words, word_scores, topic_nums = model.get_topics()

In [9]:
import numpy as np
from evaluation import get_top_words_for_topics_bertopic, topic_diversity

topics_representations = topic_words
#topics_representations = get_top_words_for_topics_bertopic(topic_model, 25)


### calculate topic diversity

In [11]:
topics_representations.shape

(55, 50)

In [12]:


topic_diversity(topics_representations, 25)

0.76

### calcualte topic coherence

In [13]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

def topic_coherence(trained_model, topic_top_words, top_k_words=25):
    if topic_top_words is None:
        return -1
    
    if top_k_words > len(topic_top_words[0]):
        print("top_k_words is larger than the number of words in the topic")
        return -1

    coherence_model_lda = CoherenceModel(
        topics=topic_top_words,
        texts=list(df['lemmatized_text']),
        dictionary=Dictionary(list(df['lemmatized_text'])),
        coherence='c_npmi',
        topn=top_k_words)

    return coherence_model_lda.get_coherence()

topic_coherence(None, topics_representations, top_k_words=10)

0.1300551227167047

## top2vec on tweets data

In [23]:
import pandas as pd
ROOT_PATH = '/home/jhladnik/'

NUM_SAMPLES = 50000
df = pd.read_parquet(
    f'{ROOT_PATH}/data/sl-tweets/df_tweets_lemmas_{NUM_SAMPLES}.parquet.gzip')


In [24]:
df["lemmatized_text_string"] = df.lemmatized_text.apply(lambda x: " ".join(x))

In [25]:
%%time
# 3 min on remote  for 5k samples
model = Top2Vec(documents=list(df["lemmatized_text_string"]), speed="learn", workers=24)


2023-01-24 20:31:49,856 - top2vec - INFO - Pre-processing documents for training
2023-01-24 20:31:51,284 - top2vec - INFO - Creating joint document/word embedding
2023-01-24 20:38:10,869 - top2vec - INFO - Creating lower dimension embedding of documents
2023-01-24 20:38:44,661 - top2vec - INFO - Finding dense areas of documents
2023-01-24 20:38:48,398 - top2vec - INFO - Finding topics


CPU times: user 25min 11s, sys: 4min 12s, total: 29min 23s
Wall time: 6min 59s


In [26]:
topic_words, word_scores, topic_nums = model.get_topics()

In [27]:
import numpy as np
from evaluation import get_top_words_for_topics_bertopic, topic_diversity

topics_representations = topic_words
#topics_representations = get_top_words_for_topics_bertopic(topic_model, 25)


### calculate topic diversity

In [28]:
topics_representations.shape

(144, 50)

In [29]:


topic_diversity(topics_representations, 25)

0.24805555555555556

### calcualte topic coherence

In [30]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

def topic_coherence(trained_model, topic_top_words, top_k_words=25):
    if topic_top_words is None:
        return -1
    
    if top_k_words > len(topic_top_words[0]):
        print("top_k_words is larger than the number of words in the topic")
        return -1

    coherence_model_lda = CoherenceModel(
        topics=topic_top_words,
        texts=list(df['lemmatized_text']),
        dictionary=Dictionary(list(df['lemmatized_text'])),
        coherence='c_npmi',
        topn=top_k_words)

    return coherence_model_lda.get_coherence()

topic_coherence(None, topics_representations, top_k_words=10)

-0.41373743560966486